In [1]:
from sec_edgar_downloader import Downloader

In [15]:
dl = Downloader("BUFN403", "sashvenk@terpmail.umd.edu")

dl.get("10-K", "AAPL")

28

In [9]:
import datetime
import pandas as pd

In [12]:
print(pd.to_datetime("20170801163015").date())

2017-08-01


In [13]:
# import libraries
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer


# download nltk corpus (first time only)
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/sashwat/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/sashwat/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/sashwat/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/sashwat/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/sashwat/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     /Users/sashwat/nltk_data...

True

In [20]:
with open("./sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt", 'r', encoding='utf-8') as file:
    raw_10k = file.read()

#print(text_content)

#analyzer = SentimentIntensityAnalyzer()
#scores = analyzer.polarity_scores(text_content)
#print(scores)

In [19]:
import requests

from bs4 import BeautifulSoup

import re

In [23]:
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

### Type filter is interesting, it looks for <TYPE> with Not flag as new line, ie terminare there, with + sign
### to look for any char afterwards until new line \n. This will give us <TYPE> followed Section Name like '10-K'
### Once we have have this, it returns String Array, below line will with find content after <TYPE> ie, '10-K' 
### as section names
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = raw_10k[doc_start:doc_end]

document['10-K'][0:500]

regex = re.compile(r'(>Item(\s|&#160;|&nbsp;)(1A|1B|7A|7|8)\.{0,1})|(ITEM\s(1A|1B|7A|7|8))')

# Use finditer to math the regex
matches = regex.finditer(document['10-K'])

# Write a for loop to print the matches
#for match in matches:
#    print(match)


matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df.head()

,item,start,end
0,>item 1a.,37821,37830
1,>item 1b.,38844,38853
2,>item 7.,45612,45620
3,>item 7a.,46739,46748
4,>item 8.,47809,47817


In [26]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)

# display the dataframe
test_df.head()

# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')

# Display the dataframe
#pos_dat

pos_dat.set_index('item', inplace=True)

# display the dataframe
pos_dat

,start,end
item,,
item1a,123704,123713
item1b,196863,196872
item7,364642,364650
item7a,792204,792213
item8,804000,804008


In [29]:
item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item1b']]
item_1a_content = BeautifulSoup(item_1a_raw, 'lxml')
#print(item_1a_content.prettify()[0:1000])
print(item_1a_content.get_text("\n\n")[0:1500])

>Item 1A. 

Risk Factors

The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item 7, “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and the consolidated financial statements and related notes in Part II, Item 8, “Financial Statements and Supplementary Data” of this Form 10-K.

The business, financial condition and operating results of the Company can be affected by a number of factors, whether currently known or unknown, including but not limited to those described below, any one or more of which could, directly or indirectly, cause the Company’s actual financial condition and operating results to vary materially from past, or from anticipated future, financial condition and operating results. Any of these factors, in whole or in part, could materially and advers

In [33]:
item_1b_raw = document['10-K'][pos_dat['start'].loc['item1b']:pos_dat['start'].loc['item7']]
item_1b_content = BeautifulSoup(item_1b_raw, 'lxml')
#print(item_1a_content.prettify()[0:1000])
print(item_1b_content.get_text("\n\n"))

>Item 1B. 

Unresolved Staff Comments

None.

Apple Inc. | 2017 Form 10-K | 

16

Item 2. 

Properties

The Company’s headquarters are located in Cupertino, California. As of 

September 30, 2017

, the Company owned 13.4 million square feet and leased 23.0 million square feet of building space, primarily in the U.S. Additionally, the Company owned a total of 4,928 acres of land, primarily in the U.S.

As of 

September 30, 2017

, the Company owned facilities and land for corporate functions, R&D and data centers at various locations throughout the U.S. Outside the U.S., the Company owned additional facilities and land for various purposes. 

The Company believes its existing facilities and equipment, which are used by all operating segments, are in good operating condition and are suitable for the conduct of its business. The Company has invested in internal capacity and strategic relationships with outside manufacturing vendors and continues to make investments in capital equipment 

In [46]:
item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item8']]
item_7_content = BeautifulSoup(item_7_raw, 'lxml')
#print(item_1a_content.prettify()[0:1000])
print(item_7_content.get_text("\n\n"))

>Item 7. 

Management’s Discussion and Analysis of Financial Condition and Results of Operations

This section and other parts of this Annual Report on Form 10-K (“Form 10-K”) contain forward-looking statements, within the meaning of the Private Securities Litigation Reform Act of 1995, that involve risks and uncertainties. Forward-looking statements provide current expectations of future events based on certain assumptions and include any statement that does not directly relate to any historical or current fact. Forward-looking statements can also be identified by words such as “future,”  “anticipates,”  “believes,”  “estimates,”  “expects,”  “intends,”  “plans,”  “predicts,”  “will,”  “would,”  “could,”  “can,”  “may,” and similar terms. Forward-looking statements are not guarantees of future performance and the Company’s actual results may differ significantly from the results discussed in the forward-looking statements. Factors that might cause such differences include, but are not

In [47]:
analyzer = SentimentIntensityAnalyzer()
scores = analyzer.polarity_scores(item_7_content.get_text("\n\n"))
print(scores)

{'neg': 0.061, 'neu': 0.824, 'pos': 0.115, 'compound': 1.0}


In [48]:
print(analyzer.polarity_scores(item_1a_content.get_text("\n\n")))

{'neg': 0.108, 'neu': 0.769, 'pos': 0.122, 'compound': 0.9985}


In [49]:
print(analyzer.polarity_scores(item_1b_content.get_text("\n\n")))

{'neg': 0.02, 'neu': 0.895, 'pos': 0.085, 'compound': 0.997}


In [55]:
#print()
f = open("myfile.txt", "w")
f.write(document['10-K'][804000:])
f.close()

In [57]:
item8 = BeautifulSoup(document['10-K'][804000:], 'lxml')
print(item8.get_text("\n\n"))

>Item 8. 

Financial Statements and Supplementary Data

Index to Consolidated Financial Statements

 

Page

Consolidated Statements of Operations for the years ended September 30, 2017, September 24, 2016 and September 26, 2015

 

39

Consolidated Statements of Comprehensive Income for the years ended September 30, 2017, September 24, 2016 and September 26, 2015

 

40

Consolidated Balance Sheets as of September 30, 2017 and September 24, 2016

 

41

Consolidated Statements of Shareholders’ Equity for the years ended September 30, 2017, September 24, 2016 and September 26, 2015

 

42

Consolidated Statements of Cash Flows for the years ended September 30, 2017, September 24, 2016 and

 

September 26, 2015

 

43

Notes to Consolidated Financial Statements

 

44

Selected Quarterly Financial Information (Unaudited)

 

68

Reports of Ernst & Young LLP, Independent Registered Public Accounting Firm

 

70

All financial statement schedules have been omitted, since the required inf

In [58]:
print(analyzer.polarity_scores(item8.get_text("\n\n")))

{'neg': 0.034, 'neu': 0.845, 'pos': 0.12, 'compound': 1.0}


In [60]:
import pysentiment2

In [75]:
#pysentiment2.LM.tokenize(item_7_content.get_text("\n\n"))
lm = pysentiment2.LM()

tokens = lm.tokenize(item_7_content.get_text("\n\n"))

score = lm.get_score(tokens)
score

{'Positive': 103,
 'Negative': 201,
 'Polarity': -0.32236841999220917,
 'Subjectivity': 0.06046141605798301}

In [76]:
scores = analyzer.polarity_scores(item_7_content.get_text("\n\n"))
scores

{'neg': 0.061, 'neu': 0.824, 'pos': 0.115, 'compound': 1.0}